# Augmenting cropped digits  
This notebook is used to create augmented pictures based on original, cropped pictures of digits. 

In [0]:
# mounting on Google Drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [0]:
# load all necessary packages

from imgaug import augmenters as iaa
from matplotlib.pyplot import imshow
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob

%matplotlib inline

In [0]:
# function for drawing random lines on images

def img_func(images, random_state, parents, hooks):
    k = np.random.randint(1,59)

    for img in images:
        img[k] = 255
        img[k+1] = 255

    return images

def keypoint_func(keypoints_on_images, random_state, parents, hooks):
    return keypoints_on_images

In [0]:
# defining augmentation sequences

seq = iaa.Sequential([
    iaa.Lambda(img_func, keypoint_func),
    iaa.GaussianBlur(sigma=(1.0, 8.0)), 
    iaa.AdditiveGaussianNoise(scale=0.1*255),
    iaa.CoarseDropout(p=(0.03, 0.05), size_percent=0.1)
])

seq2 = iaa.Sequential([
    iaa.Affine(rotate=(-10,10),
               shear=(-10,10))
])

In [0]:
def augment_digits(files, output_dir, n_augmentations, nondigits=False):
  
  for idx, file in enumerate(files):
    # loading image
    img = Image.open(file, 'r')
    img = np.array(img)
    #img = cv2.bitwise_not(np.asarray(img))

    for t in range(1, n_augmentations):

      # augmenting image
      augimg = seq(images=np.asarray(img))
      #gray = cv2.cvtColor(augimg, cv2.COLOR_BGR2GRAY)
      auggray = seq2(images=augimg)

      label = file.split(".")[-2][-1]

      # write augmented image to Drive 
      if nondigits:
        new_file_name = str(idx) + str(t) + "_n.jpg"
      else:
        new_file_name = file.split("/")[-1].replace(".", "_augmented_{}.".format(t)) 

      cv2.imwrite(output_dir + new_file_name, auggray)

**Analog cropped digits**

In [0]:
subdirs = glob.glob("/content/drive/My Drive/Team Bergere/Optimizer/Data/optimizer/HQ_analog/Split/Train_proc/*")
subdirs = [x for x in subdirs if len(x.split("/")[-1])==1]

to_augment = []
for subdir in subdirs: 
  all_files = glob.glob(subdir + "/*.jpg")
  orig_files = [x for x in all_files if "augmented" not in x]
  to_augment.append(orig_files)

In [0]:
for files in to_augment:
  digit = files[0].split("/")[-2]
  n_augmentations = 500 // len(files)
  output_dir = "/content/drive/My Drive/Team Bergere/Optimizer/Data/optimizer/HQ_analog/Split/Train_proc/_aug_digits_/" + digit + "/"
  
  augment_digits(files, output_dir, n_augmentations)

**Digital cropped digits**

In [0]:
# defining augmentation sequences

seq = iaa.Sequential([
    #iaa.Lambda(img_func, keypoint_func),
    #iaa.GaussianBlur(sigma=(1.0, 8.0)), 
    #iaa.AdditiveGaussianNoise(scale=0.1*255),
    iaa.CoarseDropout(p=(0.03, 0.05), size_percent=0.1)
])

seq2 = iaa.Sequential([
    iaa.Affine(rotate=(-10,10),
               shear=(-10,10))
])

In [0]:
subdirs = glob.glob("/content/drive/My Drive/Team Bergere/Optimizer/Data/optimizer/HQ_digital/Split_Gray/Train_set/*")
subdirs = [x for x in subdirs if len(x.split("/")[-1])==1]

to_augment = []
for subdir in subdirs: 
  all_files = glob.glob(subdir + "/*.jpg")
  orig_files = [x for x in all_files if "augmented" not in x]
  to_augment.append(orig_files)

In [0]:
for files in to_augment:
  digit = files[0].split("/")[-2]
  n_augmentations = 500 // len(files)
  output_dir = "/content/drive/My Drive/Team Bergere/Optimizer/Data/optimizer/HQ_digital/Split_Gray/Train_set/aug_digits/" + digit + "/"
  
  augment_digits(files, output_dir, n_augmentations)